# Abstract

Try out map/reduce type work on associations

# Environment

In [35]:
from functools import partial

In [1]:
from jwst.associations.tests.helpers import (
    combine_pools, 
    registry_level3_only, 
    registry_level2_only
)
from jwst.associations import generate

# Library

In [59]:
def map_rules(member, rules=None, allow=None):
    """Map member to rules"""
    asns, reprocess_list = rules.match(member)
    for reprocess in reprocess_list:
        for member in reprocess.members:
            asns.extend(map_rules(member, rules, allow=reprocess.allowed_rules))
    return asns

# Main

## Try out some simply map/reduce stuff

In [2]:
def fn(x):
    return x**2

assert fn(2) == 4

In [3]:
map(fn, range(5))

In [4]:
[x for x in map(fn, range(5))]

[0, 1, 4, 9, 16]

In [5]:
def fn_m(x):
    if x <= 10:
        return x**2
    else:
        return x, x

In [6]:
[x for x in map(fn_m, range(0, 20, 5))]

[0, 25, 100, (15, 15)]

In [7]:
def fn_y(x):
    yield x**2

In [8]:
[x for x in fn_y(2)]

[4]

In [9]:
[x for x in map(fn_y, range(5))]

[<generator object fn_y at 0x10d6e0410>,
 <generator object fn_y at 0x10d6e06d0>,
 <generator object fn_y at 0x10d6e0518>,
 <generator object fn_y at 0x10d6e0468>,
 <generator object fn_y at 0x10d6e0830>]

In [10]:
[
    x
    for y in map(fn_y, range(5))
    for x in y
]

[0, 1, 4, 9, 16]

In [11]:
def fn_ym(x):
    if x <= 10:
        yield x**2
    else:
        yield x
        yield x

In [12]:
[
    x
    for y in map(fn_ym, range(0, 20, 5))
    for x in y
]

[0, 25, 100, 15, 15]

## Try some association stuff

In [13]:
rules = registry_level3_only()

In [15]:
pool = combine_pools('data/mega_pool.csv')

In [22]:
rule_map = map(rules.match, pool)

In [23]:
matches = [match for match in rule_map]

In [60]:
mapper = partial(map_rules, rules=rules)
rule_map = map(mapper, pool)

In [61]:
matches = [match for match in rule_map]

In [64]:
matches

[[], [{
      "degraded_status": "No known degraded exposures in association.",
      "products": [
          {
              "name": "jw99009-a3001_t001_nircam_f090w",
              "members": [
                  {
                      "expname": "jw_00001_cal.fits",
                      "exptype": "SCIENCE",
                      "exposerr": null,
                      "asn_candidate": "[('o001', 'OBSERVATION')]"
                  }
              ]
          }
      ],
      "asn_id": "a3001",
      "asn_rule": "Asn_Image",
      "asn_pool": "mega_pool",
      "program": "99009",
      "target": "1",
      "code_version": "0.7.1.beta2",
      "asn_type": "image3",
      "version_id": null,
      "constraints": "Constraints:\n    wfsvisit: Is Invalid\n    opt_elem2: CLEAR\n    opt_elem: F090W\n    program: 99009\n    target: 1\n    exp_type: NRC_IMAGE\n    instrument: NIRCAM"
  }], [{
      "degraded_status": "No known degraded exposures in association.",
      "products": [
       